<a href="https://colab.research.google.com/github/yeahginny/Final_project/blob/main/0806_c_temple_%EC%97%B0%EA%B4%80%EC%84%B1%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
from os.path import join
import matplotlib.pyplot as plt
import datetime as dt
import re

In [ ]:
ls

sample_data/


In [ ]:
offline_raw = pd.read_excel("9._offline_total_2.xlsx")

In [ ]:
offline_raw =pd.read_excel('/content/9._offline_total.xlsx')

# Data Load
* 2023년 6,7월 포함한 데이터

In [ ]:
offline_raw.head()

,결제일,결제시간,주문 채널,결제내역,합계,상품별 할인,결제 할인,카드 결제,현금 결제,간편 결제,...,환불,환불 일시,배달팁(매출 포함x),카테고리,상품명,옵션,수량,상품별 단가,상품별 합계,결제메모
0,2022-02-07,19:28:53,셀러,샘플 캐모마일,-,-,-,-,-,-,...,4500,2022-02-07 19:29:37,-,에스프레소,샘플 캐모마일,-,1,4500,-,-
1,2022-02-10,10:03:28,셀러,(H) 니카라과 COE#1 외 2건,32500,-,-,32500,-,-,...,-,-,-,핸드드립,(I) 콜롬비아 로꼬 소르베,-,1,10500,10500,-
2,2022-02-10,10:03:28,셀러,(H) 니카라과 COE#1 외 2건,32500,-,-,32500,-,-,...,-,-,-,핸드드립,(H) 니카라과 COE#1,-,1,12000,10500,NaN
3,2022-02-10,10:03:28,셀러,(H) 니카라과 COE#1 외 2건,32500,-,-,32500,-,-,...,-,-,-,핸드드립,(H) 과테 레드 파카마라,-,1,10000,10500,NaN
4,2022-02-10,10:13:57,셀러,아이스 텐저린 라떼 외 2건,20000,-,-,20000,-,-,...,-,-,-,시그니처,아이스 텐저린 라떼,-,1,7000,7000,-


# 전처리

In [ ]:
def convert_to_nan(data):

    df = data.copy()

    columns = df.columns.to_list()
    for col in columns:
        df[col] = df[col].apply(lambda x: np.nan if x == "-" else x)

    return df


def drop_columns(data):

    df = data.copy()

    rm_columns = ["온라인 스토어", "사용 포인트", "적립 포인트", "사용 선불권", "배달팁(매출 포함x)", "결제메모", "주문 채널"]
    df = df.drop(rm_columns, axis = 1)

    return df


def date_conversion(data):

    df = data.copy()

    df["결제일시"] = df.loc[:, "결제일"] + " " + df.loc[:, "결제시간"]
    df["결제일시"] = pd.to_datetime(df["결제일시"])
    df["year"] = df["결제일시"].apply(lambda x: x.year)
    df["month"] = df["결제일시"].apply(lambda x: x.month)
    df["day"] = df["결제일시"].apply(lambda x: x.day)
    df["hour"] = df["결제일시"].apply(lambda x: x.hour)
    df["day_name"] = df["결제일시"].apply(lambda x: x.day_name())
    df["year_month"] = pd.to_datetime(df["결제일"]).dt.strftime("%Y-%m")

    return df


def add_weekend(data):

    df = data.copy()

    # 평일, 주말 구분
    # 0 = 평일, 1 = 주말
    df["is_weekend"] = df["day_name"].apply(lambda x: 1 if (x == "Sunday") | (x == "Saturday") else 0)

    return df


def add_season(data):

    df = data.copy()

    # 계절 추가하기
    # 봄(3~5월) = 1, 여름(6~8월) = 2, 가을(9~11월0) = 3, 겨울(12~2월) = 4
    seasons = [1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 1]
    season_dict = dict(zip(range(1,13), seasons))
    df["season"] = df["month"].map(season_dict)

    return df


def add_holiday(data):

    df = data.copy()

    conditionlist = [
        (df['결제일'] == '2022-03-01') | (df['결제일'] == '2022-05-05') | (df['결제일'] == '2022-05-08') |
        (df['결제일'] == '2022-06-06') | (df['결제일'] == '2022-08-15') | (df['결제일'] == '2022-09-09') |
        (df['결제일'] == '2022-09-10') | (df['결제일'] == '2022-09-11') | (df['결제일'] == '2022-10-09') |
        (df['결제일'] == '2022-10-03') | (df['결제일'] == '2022-12-25') |
        (df['결제일'] == '2023-01-01') | (df['결제일'] == '2023-01-21') | (df['결제일'] == '2023-01-22') |
        (df['결제일'] == '2023-01-23') | (df['결제일'] == '2023-03-01') | (df['결제일'] == '2023-05-05') |
        (df['결제일'] == '2023-05-26') | (df['결제일'] == '2023-06-06')]

    choicelist = [1]
    df['is_holiday'] = np.select(conditionlist, choicelist, default= 0)
    df["weekend_n_holiday"] = df["is_weekend"] + df["is_holiday"]

    return df

def drop_row(data):

    df = data.copy()
    df["상품명"] = df["상품명"].apply(lambda x: re.sub(r"\s", "", x))

    drop_lst = ['야외',
                '포장',
                '무료시음권',
                '캐리어',
                '종이백',
                '포크',
                '⚪️',
                '⚪', # 위 emoji 와 별개
                '일회용컵',
               ]

    custom_lst = ['덜달게',
                  '1샷추가',
                  '오틀리',
                  '연하게',
                  '시럽',
                  '얼음적게',
                  '오트사이드',
                  '물적게',
                  '바닐라시럽',
                  '2샷추가',
                  '얼음X',
                  '샷추가',
                 ]

    idx = df.loc[df["상품명"].str.contains("|".join(drop_lst)),"상품명"].index
    df = df.drop(idx, axis = 0).reset_index(drop = True)

    idx = df.loc[df["상품명"].str.contains("|".join(custom_lst)),"상품명"].index
    df.loc[idx, "카테고리"] = "커스텀"

    return df


In [ ]:
def preprocess_productname(data):
    tmp = data.copy()
    tmp["카테고리"] = tmp["카테고리"].apply(lambda x: re.sub(r"\s", "", x))
    pattern = r'\s*_\s*'


    # 카테고리 - basic_ice
    tmp.loc[tmp["카테고리"] == "Basic_ice", "상품명"] = tmp.loc[tmp["카테고리"] == "Basic_ice", "상품명"].apply(lambda x : re.sub(pattern, r'_', x))
    tmp.loc[tmp["카테고리"] == "Basic_ice", "상품명"] = tmp.loc[tmp["카테고리"] == "Basic_ice", "상품명"].apply(lambda x : re.sub(r"\s", "", x))
    tmp.loc[tmp["카테고리"] == "Basic_ice", "상품명"] = tmp.loc[tmp["카테고리"] == "Basic_ice", "상품명"].apply(lambda x : re.sub(r"플랫_", "플랫화이트_", x))
    tmp.loc[tmp["카테고리"] == "Basic_ice", "상품명"] = tmp.loc[tmp["카테고리"] == "Basic_ice", "상품명"].apply(lambda x : re.sub(r"템플", "I", x))

    beans_lst = ["클래식_", "쥬시_", "싱글_", "디카프_", "게이샤_"]
    for bean in beans_lst:
        idx = tmp.loc[tmp["카테고리"] == "Basic_ice", "상품명"][tmp.loc[tmp["카테고리"] == "Basic_ice", "상품명"].str.contains(bean)].index
        tmp.loc[idx,"상품명"] = tmp.loc[idx,"상품명"].apply(lambda x: x[len(bean):] + "_" + bean[:-1])


    # 카테고리 - basic
    tmp.loc[tmp["카테고리"] == "Basic", "상품명"] = tmp.loc[tmp["카테고리"] == "Basic", "상품명"].apply(lambda x : re.sub(pattern, r'_', x))
    tmp.loc[tmp["카테고리"] == "Basic", "상품명"] = tmp.loc[tmp["카테고리"] == "Basic", "상품명"].apply(lambda x : re.sub(r"\s|\(H\)", "", x))
    for bean in beans_lst:
        idx = tmp.loc[tmp["카테고리"] == "Basic", "상품명"][tmp.loc[tmp["카테고리"] == "Basic", "상품명"].str.contains(bean)].index
        tmp.loc[idx,"상품명"] = tmp.loc[idx,"상품명"].apply(lambda x: x[len(bean):] + "_" + bean[:-1])

    idx = tmp.loc[tmp["카테고리"] == "Basic", "상품명"][tmp.loc[tmp["카테고리"] == "Basic", "상품명"].str.contains("아메리카노|카페라떼|플랫화이트|카푸치노|바닐라라떼")].index
    tmp.loc[idx, "상품명"] = tmp.loc[idx, "상품명"].apply(lambda x: "(H)"+x)


    # 카테고리 - 시그니처
    tmp.loc[tmp["카테고리"] == "시그니처", "상품명"] = tmp.loc[tmp["카테고리"] == "시그니처", "상품명"].apply(lambda x : re.sub(r"\s", "", x))
    tmp.loc[tmp["카테고리"] == "시그니처", "상품명"] = tmp.loc[tmp["카테고리"] == "시그니처", "상품명"].apply(lambda x : re.sub(r"아이스텐라", "아이스텐저린라떼", x))
    tmp.loc[tmp["카테고리"] == "시그니처", "상품명"] = tmp.loc[tmp["카테고리"] == "시그니처", "상품명"].apply(lambda x : re.sub(r"유자아메리카노|아이스유자아메리카노", "아이스유자아메리카노", x))

    for bean in beans_lst:
        idx = tmp.loc[tmp["카테고리"] == "시그니처", "상품명"][tmp.loc[tmp["카테고리"] == "시그니처", "상품명"].str.contains(bean)].index
        tmp.loc[idx,"상품명"] = tmp.loc[idx,"상품명"].apply(lambda x: x[len(bean):] + "_" + bean[:-1])

    tmp.loc[tmp["상품명"] == "텐저린카푸치노", "상품명"] = "텐저린카푸치노_쥬시"
    tmp.loc[tmp["상품명"] == "아이스텐저린라떼", "상품명"] = "아이스텐저린라떼_쥬시"
    tmp.loc[tmp["상품명"] == "아이스유자아메리카노", "상품명"] = "아이스유자아메리카노_쥬시"

    # 카테고리 - beverage
    tmp.loc[tmp["카테고리"] == "비버리지", "상품명"] = tmp.loc[tmp["카테고리"] == "비버리지", "상품명"].apply(lambda x : re.sub(r"\s", "", x))

    rename_dict = {"차가운어린이우유": "(I)어린이우유",
                   "따뜻한어린이우유": "(H)어린이우유",

                   "얼그레이밀크티": "(H)얼그레이밀크티",

                   "제주유기농귤피주스" : "(I)제주유기농귤피주스",
                   "문경선암리사과주스" : "(I)문경선암리사과주스",
                   "제주유기농감귤주스" : "(I)제주유기농감귤주스",
                   "어린이감귤주스" : "(I)어린이감귤주스",

                   "시나몬플럼" : "(H)시나몬플럼",
                   "트로피칼루이보스" : "(H)트로피칼루이보스",
                   "카모마일" : "(H)카모마일"
                  }

    tmp.loc[tmp["카테고리"] == "비버리지", "상품명"] = tmp.loc[tmp["카테고리"] == "비버리지", "상품명"].apply(lambda x: rename_dict[x] if x in rename_dict.keys() else x)
    tmp.loc[tmp["카테고리"] == "비버리지", "상품명"].value_counts()


    # 카테고리 - 디저트
    tmp.loc[tmp["카테고리"] == "디저트", "상품명"] = tmp.loc[tmp["카테고리"] == "디저트", "상품명"].apply(lambda x : re.sub(r"\s", r'_', x))


    # 카테고리 - 블랜딩원두
    tmp.loc[tmp["카테고리"] == "블렌딩원두", "상품명"] = tmp.loc[tmp["카테고리"] == "블렌딩원두", "상품명"].apply(lambda x : re.sub(pattern, r'_', x))
    tmp.loc[tmp["카테고리"] == "블렌딩원두", "상품명"] = tmp.loc[tmp["카테고리"] == "블렌딩원두", "상품명"].apply(lambda x : re.sub(r"\s", r'_', x))


    # 카테고리 = 세트
    tmp.loc[tmp["카테고리"] == "세트", "상품명"] = tmp.loc[tmp["카테고리"] == "세트", "상품명"].apply(lambda x : re.sub(r"\s", '', x))
    tmp.loc[tmp["카테고리"] == "세트", "상품명"] = tmp.loc[tmp["카테고리"] == "세트", "상품명"].apply(lambda x : re.sub("Set.", "", x))

    for bean in beans_lst:
        idx = tmp.loc[tmp["카테고리"] == "세트", "상품명"][tmp.loc[tmp["카테고리"] == "세트", "상품명"].str.contains(bean)].index
        tmp.loc[idx,"상품명"] = tmp.loc[idx,"상품명"].apply(lambda x: x[len(bean):] + "_" + bean[:-1])

    tmp.loc[(tmp["카테고리"] == "세트") & (~tmp["상품명"].str.contains("(I)")), "상품명"] = tmp.loc[(tmp["카테고리"] == "세트") & (~tmp["상품명"].str.contains("(I)")), "상품명"].apply(lambda x : "(H)" + x)
    tmp.loc[tmp["카테고리"] == "세트", "상품명"] = tmp.loc[tmp["카테고리"] == "세트", "상품명"].apply(lambda x: "Set_" + x)


    # 카테고리 - 드립백/캡슐
    tmp.loc[tmp["카테고리"] == "드립백/캡슐", "상품명"] = tmp.loc[tmp["카테고리"] == "드립백/캡슐", "상품명"].apply(lambda x : re.sub(r"\s", r'_', x))


    # 카테고리 - 에스프레소
    tmp.loc[tmp["카테고리"] == "에스프레소", "상품명"] = tmp.loc[tmp["카테고리"] == "에스프레소", "상품명"].apply(lambda x : re.sub(pattern, r'_', x))
    tmp.loc[tmp["카테고리"] == "에스프레소", "상품명"] = tmp.loc[tmp["카테고리"] == "에스프레소", "상품명"].apply(lambda x : re.sub(r"\s", r'_', x))

    beans_lst = ["클래식_", "쥬시_", "싱글_", "디카프_", "스페셜_", "샘플_", "게이샤_"]
    for bean in beans_lst:
        idx = tmp.loc[tmp["카테고리"] == "에스프레소", "상품명"][tmp.loc[tmp["카테고리"] == "에스프레소", "상품명"].str.contains(bean)].index
        tmp.loc[idx,"상품명"] = tmp.loc[idx,"상품명"].apply(lambda x: x[len(bean):] + "_" + bean[:-1])

    # 카테고리 - 핸드드립, 싱글원두
    tmp.loc[tmp["카테고리"] == "핸드드립", "상품명"] = tmp.loc[tmp["카테고리"] == "핸드드립", "상품명"].str.strip()
    tmp.loc[tmp["카테고리"] == "핸드드립", "상품명"] = tmp.loc[tmp["카테고리"] == "핸드드립", "상품명"].apply(lambda x: re.sub(r'\)\s', ")", x))
    tmp.loc[tmp["카테고리"] == "핸드드립", "상품명"] = tmp.loc[tmp["카테고리"] == "핸드드립", "상품명"].apply(lambda x: re.sub(r'\s*:\s*', "_", x))
    tmp.loc[tmp["카테고리"] == "핸드드립", "상품명"] = tmp.loc[tmp["카테고리"] == "핸드드립", "상품명"].apply(lambda x: re.sub(r'\s+', "_", x))
    tmp.loc[tmp["카테고리"] == "핸드드립", "상품명"] = tmp.loc[tmp["카테고리"] == "핸드드립", "상품명"].apply(lambda x: re.sub(r'\(강배전\)|\(강\)', "강배전", x))
    tmp.loc[tmp["카테고리"] == "핸드드립", "상품명"] = tmp.loc[tmp["카테고리"] == "핸드드립", "상품명"].apply(lambda x: re.sub(r'\(중강배전\)|\(중\)', "중강배전", x))
    tmp.loc[tmp["카테고리"] == "핸드드립", "상품명"] = tmp.loc[tmp["카테고리"] == "핸드드립", "상품명"].apply(lambda x: re.sub(r'\(디카프\)', "디카프", x))

    tmp["상품명_원산지"] = tmp["상품명"].copy()
    idx = tmp.loc[tmp["카테고리"] == "싱글원두", "상품명_원산지"][tmp.loc[tmp["카테고리"] == "싱글원두", "상품명_원산지"].str.contains("ㅡ")].index
    tmp = tmp.drop(idx, axis = 0).reset_index(drop = True)

    rename_dict = {"디카페인 우일라 200g" : "디카페인 콜롬비아 우일라 200g",
                   "디카페인콜롬비아 리치 200g" : "디카페인 콜롬비아 리치 200g",
                   "엘리다 카투아이 100g" : "파나마 엘리다 카투아이 100g",
                   "엘리다 카투아이 ASD 100g" : "파나마 엘리다 카투아이 100g",
                   "엘파라이소 디카프 100g" : "콜롬비아 엘파라이소 디카프 100g",
                   "엘파라이소 리치 100g" : "콜롬비아 엘파라이소 리치 100g",
                   "엘파라이소 리치" : "콜롬비아 엘파라이소 리치",
                   "부산제 200g" : "르완다 부산제 200g",
                   "르완다부산제 200g" : "르완다 부산제 200g",
                   "에콰100g" : "에콰도르 100g",
                   "세로아줄 게이샤" : "콜롬비아 세로아줄 게이샤",
                   "페루게이샤" : "페루 게이샤",
                   "페루게이샤 100g" : "페루 게이샤 100g",
                   "니카라과강배전" : "니카라과 강배전",
                   "케냐키티투 200g" : "케냐 키티투 200g",
                   "케냐캄왕기. 200g" : "케냐 캄왕기 200g",
                   "쿠쿠세" : "에티오피아 쿠쿠세",
                   "(할인) 케냐카루만디 200g" : "(할인) 케냐 카루만디 200g",
                   "니카라과핀카케냐바티안" : "니카라과 핀카케냐바티안",
                   "온다라스 엘 케브라초 파라이네마 200g" : "온두라스 엘 케브라초 파라이네마 200g",
                   "페루엘사포테 200g" : "페루 엘사포테 200g",
                   "니카라과리틀 레드 200g" : "니카라과 리틀 레드 200g",
                   "(디카페인)콜롬비아 리치 200g" : "(디카페인) 콜롬비아 리치 200g",
                   "(디카페인)콜롬비아 리치 100g" : "(디카페인) 콜롬비아 리치 100g",
                   "콜룸비니 엘 파라이소 리치 100g" : "콜롬비아 엘 파라이소 리치 100g",
                   "(할인) 디카프 / 콜롬비아 엘 파라이소 리치" : "(할인) 디카프 콜롬비아 엘 파라이소 리치",
                   "[로우카페인] 시티트래블러" : "시티트래블러 로우카페인",

                   "(I)디카프_에티오피아" : "(I)에티오피아_디카프",
                   "(H)디카프_에티오피아" : "(H)에티오피아_디카프",
                   "(H)과테_레드_파카마라" : "(H)과테말라_레드_파카마라",
                   "(I)과테_레드_파카마라" : "(I)과테말라_레드_파카마라",
                   "(I)과테말라엘모리또" : "(I)과테말라_엘모리또",
                   "(H)과테말라엘모리또" : "(H)과테말라_엘모리또",
                   "(H)케냐띠리쿠" : "(H)케냐_띠리쿠",
                   "(H)콰트로_콜롬비아" : "(H)콜롬비아_콰트로",
                   "(I)콰트로_콜롬비아" : "(I)콜롬비아_콰트로",
                   "(H)디카페인_콜롬비아" : "(H)콜롬비아_디카페인",
                   "(I)디카페인_콜롬비아" : "(I)콜롬비아_디카페인",
                   "(H)디카페인_니카라과" : "(H)니카라과_디카페인",
                   "(I)디카페인_니카라과" : "(I)니카라과_디카페인",
                   "(I)디카페인_에티오피아" : "(I)에티오피아_디카페인",
                   "(H)디카페인_에티오피아" : "(H)에티오피아_디카페인",
                   "(I)오늘의커피" : "(I)오늘의_커피",
                   "(H)오늘의커피" : "(H)오늘의_커피",
                   "(H)인도네시나_프린사_콜렉티브" : "(H)인도네시아_프린사_콜렉티브",
                }

    tmp.loc[tmp["카테고리"] == "싱글원두", "상품명_원산지"] = tmp.loc[tmp["카테고리"] == "싱글원두", "상품명_원산지"].str.strip()
    tmp.loc[:, "상품명_원산지"] = tmp.loc[:, "상품명_원산지"].replace(rename_dict)

    tmp.loc[tmp["카테고리"] == "핸드드립", "상품명_원산지"] = tmp.loc[tmp["카테고리"] == "핸드드립", "상품명_원산지"].apply(lambda x : re.sub("예맨", "예멘", x))
    tmp.loc[tmp["카테고리"] == "핸드드립", "상품명_원산지"] = tmp.loc[tmp["카테고리"] == "핸드드립", "상품명_원산지"].apply(lambda x : re.sub(r"\(H\)|\(I\)", "", x))
    tmp.loc[tmp["카테고리"] == "핸드드립", "상품명_원산지"] = tmp.loc[tmp["카테고리"] == "핸드드립", "상품명_원산지"].apply(lambda x: x.split("_")[0])

    tmp.loc[tmp["카테고리"] == "싱글원두", "상품명_원산지"] = tmp.loc[tmp["카테고리"] == "싱글원두", "상품명_원산지"].apply(lambda x: re.sub(pattern, " ", x))
    tmp.loc[tmp["카테고리"] == "싱글원두", "상품명_원산지"] = tmp.loc[tmp["카테고리"] == "싱글원두", "상품명_원산지"].apply(lambda x : re.sub("[()]", "", x))
    tmp.loc[tmp["카테고리"] == "싱글원두", "상품명_원산지"] = tmp.loc[tmp["카테고리"] == "싱글원두", "상품명_원산지"].apply(lambda x : re.sub(r"_?[0-9]*g|할인|강배전", "", x))
    tmp.loc[tmp["카테고리"] == "싱글원두", "상품명_원산지"] = tmp.loc[tmp["카테고리"] == "싱글원두", "상품명_원산지"].str.strip()
    tmp.loc[tmp["카테고리"] == "싱글원두", "상품명_원산지"] = tmp.loc[tmp["카테고리"] == "싱글원두", "상품명_원산지"].apply(lambda x : re.sub(r"\s", "_", x))

    lst = ["디카페인_", "디카프_", "콰트로_"]
    for i in lst:
        tmp.loc[tmp["카테고리"] == "싱글원두", "상품명_원산지"] = tmp.loc[tmp["카테고리"] == "싱글원두", "상품명_원산지"].apply(lambda x: x[len(i):] + "_" + i[:-1] if x[:len(i)] == i else x)
    tmp.loc[tmp["카테고리"] == "싱글원두", "상품명_원산지"] = tmp.loc[tmp["카테고리"] == "싱글원두", "상품명_원산지"].apply(lambda x: x.split("_")[0])


    return tmp

In [ ]:
# pd.options.display.max_columns = None

# offline_raw = pd.read_excel(join(path, "9. offline_total.xlsx"))
offline_df = convert_to_nan(offline_raw)

offline_df = drop_columns(offline_df)
offline_df = date_conversion(offline_df)
offline_df = add_weekend(offline_df)
offline_df = add_season(offline_df)
offline_df = add_holiday(offline_df)
offline_df = preprocess_productname(offline_df)
offline_df = drop_row(offline_df)

offline_df

<ipython-input-7-91640ed3e34d>:82: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  tmp.loc[(tmp["카테고리"] == "세트") & (~tmp["상품명"].str.contains("(I)")), "상품명"] = tmp.loc[(tmp["카테고리"] == "세트") & (~tmp["상품명"].str.contains("(I)")), "상품명"].apply(lambda x : "(H)" + x)


,결제일,결제시간,결제내역,합계,상품별 할인,결제 할인,카드 결제,현금 결제,간편 결제,기타 결제,...,month,day,hour,day_name,year_month,is_weekend,season,is_holiday,weekend_n_holiday,상품명_원산지
0,2022-02-07,19:28:53,샘플 캐모마일,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2,7,19,Monday,2022-02,0,1,0,0,캐모마일_샘플
1,2022-02-10,10:03:28,(H) 니카라과 COE#1 외 2건,32500.0,NaN,NaN,32500.0,NaN,NaN,NaN,...,2,10,10,Thursday,2022-02,0,1,0,0,콜롬비아
2,2022-02-10,10:03:28,(H) 니카라과 COE#1 외 2건,32500.0,NaN,NaN,32500.0,NaN,NaN,NaN,...,2,10,10,Thursday,2022-02,0,1,0,0,니카라과
3,2022-02-10,10:03:28,(H) 니카라과 COE#1 외 2건,32500.0,NaN,NaN,32500.0,NaN,NaN,NaN,...,2,10,10,Thursday,2022-02,0,1,0,0,과테말라
4,2022-02-10,10:13:57,아이스 텐저린 라떼 외 2건,20000.0,NaN,NaN,20000.0,NaN,NaN,NaN,...,2,10,10,Thursday,2022-02,0,1,0,0,아이스텐저린라떼_쥬시
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158747,2023-07-31,16:59:01,클래식_(I) 카페라떼 외 1건,13500.0,NaN,NaN,13500.0,NaN,NaN,NaN,...,7,31,16,Monday,2023-07,0,3,0,0,아이스텐저린라떼_쥬시
158748,2023-07-31,17:17:57,쥬시_아이스 텐라,21000.0,NaN,NaN,21000.0,NaN,NaN,NaN,...,7,31,17,Monday,2023-07,0,3,0,0,아이스텐저린라떼_쥬시
158749,2023-07-31,17:19:12,쥬시_유자 아메리카노 외 1건,14000.0,NaN,NaN,14000.0,NaN,NaN,NaN,...,7,31,17,Monday,2023-07,0,3,0,0,아이스유자아메리카노_쥬시
158750,2023-07-31,17:19:12,쥬시_유자 아메리카노 외 1건,14000.0,NaN,NaN,14000.0,NaN,NaN,NaN,...,7,31,17,Monday,2023-07,0,3,0,0,아이스텐저린라떼_쥬시


# apriori

## 1년치
* 2022-07-01 ~ 2023-07-31

In [ ]:
offline_year = offline_df[offline_df['결제일'].between('2022-07-01', '2023-07-31')].copy()
offline_year.tail()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,결제일,결제시간,결제내역,합계,상품별 할인,결제 할인,카드 결제,현금 결제,간편 결제,기타 결제,...,month,day,hour,day_name,year_month,is_weekend,season,is_holiday,weekend_n_holiday,상품명_원산지
158747,2023-07-31,16:59:01,클래식_(I) 카페라떼 외 1건,13500.0,NaN,NaN,13500.0,NaN,NaN,NaN,...,7,31,16,Monday,2023-07,0,3,0,0,아이스텐저린라떼_쥬시
158748,2023-07-31,17:17:57,쥬시_아이스 텐라,21000.0,NaN,NaN,21000.0,NaN,NaN,NaN,...,7,31,17,Monday,2023-07,0,3,0,0,아이스텐저린라떼_쥬시
158749,2023-07-31,17:19:12,쥬시_유자 아메리카노 외 1건,14000.0,NaN,NaN,14000.0,NaN,NaN,NaN,...,7,31,17,Monday,2023-07,0,3,0,0,아이스유자아메리카노_쥬시
158750,2023-07-31,17:19:12,쥬시_유자 아메리카노 외 1건,14000.0,NaN,NaN,14000.0,NaN,NaN,NaN,...,7,31,17,Monday,2023-07,0,3,0,0,아이스텐저린라떼_쥬시
158751,2023-07-31,18:18:25,원두 별도구매,100000.0,NaN,NaN,100000.0,NaN,NaN,NaN,...,7,31,18,Monday,2023-07,0,3,0,0,원두 별도구매


### 휘낭시에 쌍포함

In [ ]:
menu_lst = [
            '시그니처',
            # 'Basic_ice',
            # '에스프레소',
            # 'Basic',
            #  '비버리지',
            #'핸드드립',
            # '드립백/캡슐',
            # '싱글원두',
            # '블렌딩원두',
            # '커스텀',
             '디저트',
            # "세트",
           ]
menu_df = offline_year.loc[offline_year["카테고리"].str.contains("|".join(menu_lst)),:].reset_index(drop = True)
menu_df.head()

,결제일,결제시간,결제내역,합계,상품별 할인,결제 할인,카드 결제,현금 결제,간편 결제,기타 결제,...,month,day,hour,day_name,year_month,is_weekend,season,is_holiday,weekend_n_holiday,상품명_원산지
0,2022-07-01,10:01:48,(I) 파나마 잔슨 게이샤 외 2건,25000.0,NaN,NaN,25000.0,NaN,NaN,NaN,...,7,1,10,Friday,2022-07,0,3,0,0,아이스텐저린라떼_쥬시
1,2022-07-01,10:12:25,바스크 치즈케이크 외 2건,21000.0,NaN,NaN,21000.0,NaN,NaN,NaN,...,7,1,10,Friday,2022-07,0,3,0,0,바스크_치즈케이크
2,2022-07-01,10:12:25,바스크 치즈케이크 외 2건,21000.0,NaN,NaN,21000.0,NaN,NaN,NaN,...,7,1,10,Friday,2022-07,0,3,0,0,텐저린카푸치노_쥬시
3,2022-07-01,10:35:57,슈퍼클린 _ 쥬시 외 1건,13500.0,NaN,NaN,13500.0,NaN,NaN,NaN,...,7,1,10,Friday,2022-07,0,3,0,0,아이스텐저린라떼_쥬시
4,2022-07-01,10:42:52,아이스 텐저린 라떼 외 1건,13500.0,NaN,NaN,13500.0,NaN,NaN,NaN,...,7,1,10,Friday,2022-07,0,3,0,0,아이스텐저린라떼_쥬시


In [ ]:
records = menu_df[["결제일시", "상품명"]]

records = records.groupby("결제일시").agg({"상품명" : lambda x : list(x)}).reset_index(drop = True)

records = records[records["상품명"].apply(lambda x : True if len(x) > 1 else False)]

records

,상품명
1,"[바스크_치즈케이크, 텐저린카푸치노_쥬시]"
4,"[아이스텐저린라떼_쥬시, 바스크_치즈케이크, 텐저린카푸치노_쥬시]"
7,"[텐저린카푸치노_쥬시, 아이스유자아메리카노_쥬시]"
8,"[아이스텐저린라떼_쥬시, 아이스유자아메리카노_쥬시]"
14,"[아이스텐저린라떼_쥬시, 아이스유자아메리카노_쥬시]"
...,...
29030,"[아이스텐저린라떼_쥬시, 텐저린카푸치노_쥬시]"
29034,"[아이스유자아메리카노_쥬시, 헤이즐넛_휘낭시에, 플레인_휘낭시에]"
29035,"[아이스텐저린라떼_쥬시, 아이스유자아메리카노_쥬시, 텐저린카푸치노_쥬시]"
29041,"[아이스유자아메리카노_쥬시, 바스크_치즈케이크]"


In [ ]:
from matplotlib.colors import LinearSegmentedColormap
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

In [ ]:
te = TransactionEncoder()
te_ary = te.fit_transform(records["상품명"])
te_df = pd.DataFrame(te_ary, columns= te.columns_)
te_df

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,딥초코쿠키,레몬_휘낭시에,무화과_휘낭시에,바스크_치즈케이크,브라우니,시나몬_휘낭시에,아이스유자아메리카노_디카프,아이스유자아메리카노_쥬시,아이스텐저린라떼_디카프,아이스텐저린라떼_쥬시,잠봉뵈르,텐저린카푸치노_디카프,텐저린카푸치노_쥬시,플레인_휘낭시에,헤이즐넛_휘낭시에
0,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False
1,False,False,False,True,False,False,False,False,False,True,False,False,True,False,False
2,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False
3,False,False,False,False,False,False,False,True,False,True,False,False,False,False,False
4,False,False,False,False,False,False,False,True,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10901,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False
10902,False,False,False,False,False,False,False,True,False,False,False,False,False,True,True
10903,False,False,False,False,False,False,False,True,False,True,False,False,True,False,False
10904,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False


In [ ]:
itemset = apriori(te_df,
                  min_support=0.005,
                  max_len=5,
                  use_colnames=True,
                  verbose=1,
                 )
itemset['length'] = itemset['itemsets'].map(lambda x: len(x))
itemset = itemset.sort_values(by = 'support',ascending=False)
itemset

Processing 216 combinations | Sampling itemset size 4


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,support,itemsets,length
9,0.633871,(아이스텐저린라떼_쥬시),1
7,0.454887,(아이스유자아메리카노_쥬시),1
12,0.317073,(텐저린카푸치노_쥬시),1
48,0.310563,"(아이스유자아메리카노_쥬시, 아이스텐저린라떼_쥬시)",2
3,0.193563,(바스크_치즈케이크),1
...,...,...,...
16,0.005868,"(바스크_치즈케이크, 레몬_휘낭시에)",2
64,0.005777,"(무화과_휘낭시에, 텐저린카푸치노_쥬시, 레몬_휘낭시에)",3
55,0.005777,"(텐저린카푸치노_디카프, 아이스텐저린라떼_쥬시)",2
92,0.005410,"(아이스텐저린라떼_쥬시, 플레인_휘낭시에, 헤이즐넛_휘낭시에)",3


In [ ]:
from mlxtend.frequent_patterns import association_rules
association_df = association_rules(itemset, metric="lift", min_threshold= 1)
association_df.sort_values(by = "antecedent support", ascending = False)
#lift 1보다 크고
#음료 + 디저트 함께 묶인것

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
1,(아이스텐저린라떼_쥬시),(아이스유자아메리카노_쥬시),0.633871,0.454887,0.310563,0.489946,1.077072,0.022223,1.068736,0.195443
0,(아이스유자아메리카노_쥬시),(아이스텐저린라떼_쥬시),0.454887,0.633871,0.310563,0.682725,1.077072,0.022223,1.153980,0.131271
6,(텐저린카푸치노_쥬시),(레몬_휘낭시에),0.317073,0.041995,0.016688,0.052632,1.253275,0.003373,1.011227,0.295918
66,(텐저린카푸치노_쥬시),"(무화과_휘낭시에, 레몬_휘낭시에)",0.317073,0.016138,0.005777,0.018219,1.128934,0.000660,1.002119,0.167234
61,(텐저린카푸치노_쥬시),(텐저린카푸치노_디카프),0.317073,0.017238,0.005960,0.018797,1.090426,0.000494,1.001589,0.121429
...,...,...,...,...,...,...,...,...,...,...
72,"(무화과_휘낭시에, 브라우니)",(플레인_휘낭시에),0.014487,0.140657,0.005226,0.360759,2.564826,0.003189,1.344320,0.619079
40,"(플레인_휘낭시에, 헤이즐넛_휘낭시에)",(무화과_휘낭시에),0.014212,0.145425,0.006785,0.477419,3.282935,0.004718,1.635299,0.705420
56,"(시나몬_휘낭시에, 플레인_휘낭시에)",(무화과_휘낭시에),0.012745,0.145425,0.006235,0.489209,3.364003,0.004382,1.673042,0.711807
50,"(플레인_휘낭시에, 레몬_휘낭시에)",(무화과_휘낭시에),0.012379,0.145425,0.006694,0.540741,3.718360,0.004893,1.860769,0.740227


### 휘낭시에 쌍제거

In [ ]:
menu_lst = [
             '시그니처',
            # 'Basic_ice',
            # '에스프레소',
            # 'Basic',
            # '비버리지',
            #'핸드드립',
            # '드립백/캡슐',
            # '싱글원두',
            # '블렌딩원두',
            # '커스텀',
             '디저트',
            # "세트",
           ]
menu_df = offline_year.loc[offline_year["카테고리"].str.contains("|".join(menu_lst)),:].reset_index(drop = True)
menu_df.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,결제일,결제시간,결제내역,합계,상품별 할인,결제 할인,카드 결제,현금 결제,간편 결제,기타 결제,...,month,day,hour,day_name,year_month,is_weekend,season,is_holiday,weekend_n_holiday,상품명_원산지
0,2022-07-01,10:01:48,(I) 파나마 잔슨 게이샤 외 2건,25000.0,NaN,NaN,25000.0,NaN,NaN,NaN,...,7,1,10,Friday,2022-07,0,3,0,0,아이스텐저린라떼_쥬시
1,2022-07-01,10:12:25,바스크 치즈케이크 외 2건,21000.0,NaN,NaN,21000.0,NaN,NaN,NaN,...,7,1,10,Friday,2022-07,0,3,0,0,바스크_치즈케이크
2,2022-07-01,10:12:25,바스크 치즈케이크 외 2건,21000.0,NaN,NaN,21000.0,NaN,NaN,NaN,...,7,1,10,Friday,2022-07,0,3,0,0,텐저린카푸치노_쥬시
3,2022-07-01,10:35:57,슈퍼클린 _ 쥬시 외 1건,13500.0,NaN,NaN,13500.0,NaN,NaN,NaN,...,7,1,10,Friday,2022-07,0,3,0,0,아이스텐저린라떼_쥬시
4,2022-07-01,10:42:52,아이스 텐저린 라떼 외 1건,13500.0,NaN,NaN,13500.0,NaN,NaN,NaN,...,7,1,10,Friday,2022-07,0,3,0,0,아이스텐저린라떼_쥬시


In [ ]:
records = menu_df[["결제일시", "상품명"]]

records = records.groupby("결제일시").agg({"상품명" : lambda x : list(x)}).reset_index(drop = True)

records = records[records["상품명"].apply(lambda x : True if len(x) > 1 else False)]

records

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,상품명
1,"[바스크_치즈케이크, 텐저린카푸치노_쥬시]"
4,"[아이스텐저린라떼_쥬시, 바스크_치즈케이크, 텐저린카푸치노_쥬시]"
7,"[텐저린카푸치노_쥬시, 아이스유자아메리카노_쥬시]"
8,"[아이스텐저린라떼_쥬시, 아이스유자아메리카노_쥬시]"
14,"[아이스텐저린라떼_쥬시, 아이스유자아메리카노_쥬시]"
...,...
29030,"[아이스텐저린라떼_쥬시, 텐저린카푸치노_쥬시]"
29034,"[아이스유자아메리카노_쥬시, 헤이즐넛_휘낭시에, 플레인_휘낭시에]"
29035,"[아이스텐저린라떼_쥬시, 아이스유자아메리카노_쥬시, 텐저린카푸치노_쥬시]"
29041,"[아이스유자아메리카노_쥬시, 바스크_치즈케이크]"


In [ ]:
records = records["상품명"].apply(lambda x: ",".join(x)).reset_index()
records

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,index,상품명
0,1,"바스크_치즈케이크,텐저린카푸치노_쥬시"
1,4,"아이스텐저린라떼_쥬시,바스크_치즈케이크,텐저린카푸치노_쥬시"
2,7,"텐저린카푸치노_쥬시,아이스유자아메리카노_쥬시"
3,8,"아이스텐저린라떼_쥬시,아이스유자아메리카노_쥬시"
4,14,"아이스텐저린라떼_쥬시,아이스유자아메리카노_쥬시"
...,...,...
10901,29030,"아이스텐저린라떼_쥬시,텐저린카푸치노_쥬시"
10902,29034,"아이스유자아메리카노_쥬시,헤이즐넛_휘낭시에,플레인_휘낭시에"
10903,29035,"아이스텐저린라떼_쥬시,아이스유자아메리카노_쥬시,텐저린카푸치노_쥬시"
10904,29041,"아이스유자아메리카노_쥬시,바스크_치즈케이크"


In [ ]:
# 휘낭시에 2개 이상 없애기
two_df = records[records['상품명'].str.count(r'휘낭시에') >= 2].index
records.drop(two_df,inplace=True)
records

#삭제된듯

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,index,상품명
0,1,"바스크_치즈케이크,텐저린카푸치노_쥬시"
1,4,"아이스텐저린라떼_쥬시,바스크_치즈케이크,텐저린카푸치노_쥬시"
2,7,"텐저린카푸치노_쥬시,아이스유자아메리카노_쥬시"
3,8,"아이스텐저린라떼_쥬시,아이스유자아메리카노_쥬시"
4,14,"아이스텐저린라떼_쥬시,아이스유자아메리카노_쥬시"
...,...,...
10900,29024,"아이스유자아메리카노_쥬시,바스크_치즈케이크"
10901,29030,"아이스텐저린라떼_쥬시,텐저린카푸치노_쥬시"
10903,29035,"아이스텐저린라떼_쥬시,아이스유자아메리카노_쥬시,텐저린카푸치노_쥬시"
10904,29041,"아이스유자아메리카노_쥬시,바스크_치즈케이크"


In [ ]:
#index 열 삭제
records = records.drop(['index'], axis = 1)
records

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,상품명
0,"바스크_치즈케이크,텐저린카푸치노_쥬시"
1,"아이스텐저린라떼_쥬시,바스크_치즈케이크,텐저린카푸치노_쥬시"
2,"텐저린카푸치노_쥬시,아이스유자아메리카노_쥬시"
3,"아이스텐저린라떼_쥬시,아이스유자아메리카노_쥬시"
4,"아이스텐저린라떼_쥬시,아이스유자아메리카노_쥬시"
...,...
10900,"아이스유자아메리카노_쥬시,바스크_치즈케이크"
10901,"아이스텐저린라떼_쥬시,텐저린카푸치노_쥬시"
10903,"아이스텐저린라떼_쥬시,아이스유자아메리카노_쥬시,텐저린카푸치노_쥬시"
10904,"아이스유자아메리카노_쥬시,바스크_치즈케이크"


In [ ]:
records = records["상품명"].apply(lambda x: x.split(",")).reset_index()
records

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,index,상품명
0,0,"[바스크_치즈케이크, 텐저린카푸치노_쥬시]"
1,1,"[아이스텐저린라떼_쥬시, 바스크_치즈케이크, 텐저린카푸치노_쥬시]"
2,2,"[텐저린카푸치노_쥬시, 아이스유자아메리카노_쥬시]"
3,3,"[아이스텐저린라떼_쥬시, 아이스유자아메리카노_쥬시]"
4,4,"[아이스텐저린라떼_쥬시, 아이스유자아메리카노_쥬시]"
...,...,...
9825,10900,"[아이스유자아메리카노_쥬시, 바스크_치즈케이크]"
9826,10901,"[아이스텐저린라떼_쥬시, 텐저린카푸치노_쥬시]"
9827,10903,"[아이스텐저린라떼_쥬시, 아이스유자아메리카노_쥬시, 텐저린카푸치노_쥬시]"
9828,10904,"[아이스유자아메리카노_쥬시, 바스크_치즈케이크]"


* apriori 다음코드

In [ ]:
te = TransactionEncoder()
te_ary = te.fit_transform(records["상품명"])
te_df = pd.DataFrame(te_ary, columns= te.columns_)
te_df

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,딥초코쿠키,레몬_휘낭시에,무화과_휘낭시에,바스크_치즈케이크,브라우니,시나몬_휘낭시에,아이스유자아메리카노_디카프,아이스유자아메리카노_쥬시,아이스텐저린라떼_디카프,아이스텐저린라떼_쥬시,잠봉뵈르,텐저린카푸치노_디카프,텐저린카푸치노_쥬시,플레인_휘낭시에,헤이즐넛_휘낭시에
0,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False
1,False,False,False,True,False,False,False,False,False,True,False,False,True,False,False
2,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False
3,False,False,False,False,False,False,False,True,False,True,False,False,False,False,False
4,False,False,False,False,False,False,False,True,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9825,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False
9826,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False
9827,False,False,False,False,False,False,False,True,False,True,False,False,True,False,False
9828,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False


In [ ]:
itemset = apriori(te_df,
                  min_support=0.005,
                  max_len=5,
                  use_colnames=True,
                  verbose=1,
                 )
itemset['length'] = itemset['itemsets'].map(lambda x: len(x))
itemset = itemset.sort_values(by = 'support',ascending=False)
itemset

Processing 96 combinations | Sampling itemset size 4


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,support,itemsets,length
9,0.669074,(아이스텐저린라떼_쥬시),1
7,0.485758,(아이스유자아메리카노_쥬시),1
37,0.334995,"(아이스유자아메리카노_쥬시, 아이스텐저린라떼_쥬시)",2
12,0.334690,(텐저린카푸치노_쥬시),1
3,0.202035,(바스크_치즈케이크),1
...,...,...,...
54,0.006205,"(브라우니, 바스크_치즈케이크, 텐저린카푸치노_쥬시)",3
44,0.006205,"(텐저린카푸치노_디카프, 아이스텐저린라떼_쥬시)",2
31,0.006002,"(잠봉뵈르, 브라우니)",2
15,0.005697,"(아이스유자아메리카노_쥬시, 레몬_휘낭시에)",2


In [ ]:
from mlxtend.frequent_patterns import association_rules
association_df = association_rules(itemset, metric="lift", min_threshold= 1)
association_df.sort_values(by = "antecedent support", ascending = False)
#lift 1보다 크고
#음료 + 디저트 함께 묶인것

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
1,(아이스텐저린라떼_쥬시),(아이스유자아메리카노_쥬시),0.669074,0.485758,0.334995,0.500684,1.030728,0.009987,1.029894,0.090086
0,(아이스유자아메리카노_쥬시),(아이스텐저린라떼_쥬시),0.485758,0.669074,0.334995,0.689634,1.030728,0.009987,1.066242,0.057972
4,(텐저린카푸치노_쥬시),(레몬_휘낭시에),0.334690,0.022380,0.010783,0.032219,1.439597,0.003293,1.010166,0.458975
7,(텐저린카푸치노_쥬시),(텐저린카푸치노_디카프),0.334690,0.018616,0.006511,0.019453,1.044928,0.000280,1.000853,0.064626
3,(바스크_치즈케이크),(브라우니),0.202035,0.158698,0.035504,0.175730,1.107325,0.003441,1.020663,0.121462
2,(브라우니),(바스크_치즈케이크),0.158698,0.202035,0.035504,0.223718,1.107325,0.003441,1.027932,0.115206
5,(레몬_휘낭시에),(텐저린카푸치노_쥬시),0.022380,0.334690,0.010783,0.481818,1.439597,0.003293,1.283932,0.312352
6,(텐저린카푸치노_디카프),(텐저린카푸치노_쥬시),0.018616,0.334690,0.006511,0.349727,1.044928,0.000280,1.023124,0.043812


## 계절 분류

In [ ]:
offline_season = offline_df[offline_df['season'].between(1, 4)].copy()
offline_season.tail()

In [ ]:
menu_lst = [
            # '시그니처',
            # 'Basic_ice',
            '디저트',
            # '에스프레소',
            # 'Basic',
            '비버리지',
            # '핸드드립',
            # '드립백/캡슐',
            # '싱글원두',
            # '블렌딩원두',
            # '커스텀',
            # "세트",
           ]
menu_df_se = offline_season.loc[offline_season["카테고리"].str.contains("|".join(menu_lst)),:].reset_index(drop = True)
menu_df_se

In [ ]:
records_se = menu_df[["결제일시", "카테고리", "상품명"]]

records_se = records_se.groupby("결제일시").agg({"상품명" : lambda x : list(x)}).reset_index(drop = True)

records_se = records_se[records_se["상품명"].apply(lambda x : True if len(x) > 1 else False)]

records_se

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,상품명
0,"[(I)얼그레이밀크티, 브라우니]"
29,"[(H)카모마일, 잠봉뵈르]"
40,"[(H)얼그레이밀크티, 브라우니]"
44,"[(H)트로피칼루이보스, (H)얼그레이밀크티]"
50,"[(H)카모마일, (I)얼그레이밀크티]"
...,...
21653,"[(I)제주유기농귤피주스, 바스크_치즈케이크]"
21656,"[(I)얼그레이밀크티, (I)오미자에이드, 바스크_치즈케이크]"
21658,"[(I)유자에이드, (I)제주유기농귤피주스, (H)카모마일]"
21659,"[(I)제주유기농귤피주스, 헤이즐넛_휘낭시에, 플레인_휘낭시에]"


In [ ]:
records_se = records_se["상품명"].apply(lambda x: ",".join(x))

records_se = records_se.reset_index()
records_se

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,index,상품명
0,0,"(I)얼그레이밀크티,브라우니"
1,29,"(H)카모마일,잠봉뵈르"
2,40,"(H)얼그레이밀크티,브라우니"
3,44,"(H)트로피칼루이보스,(H)얼그레이밀크티"
4,50,"(H)카모마일,(I)얼그레이밀크티"
...,...,...
5557,21653,"(I)제주유기농귤피주스,바스크_치즈케이크"
5558,21656,"(I)얼그레이밀크티,(I)오미자에이드,바스크_치즈케이크"
5559,21658,"(I)유자에이드,(I)제주유기농귤피주스,(H)카모마일"
5560,21659,"(I)제주유기농귤피주스,헤이즐넛_휘낭시에,플레인_휘낭시에"


In [ ]:
# season 뽑아서 새로 만들기
season = menu_df[["결제일시", "상품명", "season"]]

season = season.groupby("결제일시").agg({"season" : lambda x : list(x)}).reset_index(drop = True)

season = season[season["season"].apply(lambda x : True if len(x) > 1 else False)]

season = season.reset_index()

# "season" 열 내 중복값을 하나로 만들어주기
season['season'] = season['season'].apply(lambda x: ' '.join(set(map(str, x))))

season

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,index,season
0,0,1
1,29,1
2,40,1
3,44,1
4,50,1
...,...,...
5557,21653,3
5558,21656,3
5559,21658,3
5560,21659,3


In [ ]:
# 기존 records와 함께 계절을 합치기

records_se = pd.concat([records_se, season], axis = 1)
records_se

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,index,상품명,index,season
0,0,"(I)얼그레이밀크티,브라우니",0,1
1,29,"(H)카모마일,잠봉뵈르",29,1
2,40,"(H)얼그레이밀크티,브라우니",40,1
3,44,"(H)트로피칼루이보스,(H)얼그레이밀크티",44,1
4,50,"(H)카모마일,(I)얼그레이밀크티",50,1
...,...,...,...,...
5557,21653,"(I)제주유기농귤피주스,바스크_치즈케이크",21653,3
5558,21656,"(I)얼그레이밀크티,(I)오미자에이드,바스크_치즈케이크",21656,3
5559,21658,"(I)유자에이드,(I)제주유기농귤피주스,(H)카모마일",21658,3
5560,21659,"(I)제주유기농귤피주스,헤이즐넛_휘낭시에,플레인_휘낭시에",21659,3


In [ ]:
#원하는 계절 없애기
records_se = records_se[records.season != '1']
records_se = records_se[records.season != '2']
records_se = records_se[records.season != '3']

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
#index 열 삭제
records_se = records_se.drop(['index','season'], axis = 1)
records_se

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
#아포리 돌리기전에 사전 작업
records_se = records_se["상품명"].apply(lambda x: x.split(","))
records_se = records_se.reset_index()
records_se

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,index,상품명
0,1884,"[(I)얼그레이밀크티, 바스크_치즈케이크]"
1,1885,"[(I)제주유기농귤피주스, 브라우니]"
2,1886,"[플레인_휘낭시에, 무화과_휘낭시에, 브라우니]"
3,1887,"[(I)얼그레이밀크티, 브라우니]"
4,1888,"[(I)얼그레이밀크티, (I)시나몬플럼, (I)패션프루트에이드]"
...,...,...
1060,2944,"[바스크_치즈케이크, 잠봉뵈르]"
1061,2945,"[브라우니, 바스크_치즈케이크]"
1062,2946,"[바스크_치즈케이크, 브라우니]"
1063,2947,"[레몬_휘낭시에, 브라우니]"


In [ ]:
te = TransactionEncoder()
te_ary = te.fit_transform(records_se["상품명"])
te_df = pd.DataFrame(te_ary, columns= te.columns_)
te_df

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,(H)시나몬플럼,(H)어린이우유,(H)얼그레이밀크티,(H)카모마일,(H)트로피칼루이보스,(I)문경선암리사과주스,(I)시나몬플럼,(I)어린이감귤주스,(I)어린이우유,(I)얼그레이밀크티,(I)제주유기농귤피주스,(I)천혜향에이드,(I)카모마일,(I)트로피칼루이보스,(I)패션프루트에이드,레몬_휘낭시에,무화과_휘낭시에,바스크_치즈케이크,브라우니,시나몬_휘낭시에,잠봉뵈르,플레인_휘낭시에
0,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True,False,False,True
3,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False
4,False,False,False,False,False,False,True,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1060,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False
1061,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False
1062,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False
1063,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False


In [ ]:
itemset = apriori(te_df,
                  min_support=0.005,
                  max_len=5,
                  use_colnames=True,
                  verbose=1,
                 )
itemset['length'] = itemset['itemsets'].map(lambda x: len(x))
itemset = itemset.sort_values(by = 'support',ascending=False)
itemset

Processing 92 combinations | Sampling itemset size 4


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,support,itemsets,length
16,0.307042,(플레인_휘낭시에),1
12,0.307042,(바스크_치즈케이크),1
13,0.298592,(브라우니),1
11,0.292019,(무화과_휘낭시에),1
8,0.212207,((I)제주유기농귤피주스),1
...,...,...,...
95,0.005634,"((I)제주유기농귤피주스, 무화과_휘낭시에, 시나몬_휘낭시에)",3
94,0.005634,"((I)얼그레이밀크티, 시나몬_휘낭시에, 플레인_휘낭시에)",3
91,0.005634,"(브라우니, (I)문경선암리사과주스, 바스크_치즈케이크)",3
90,0.005634,"((I)문경선암리사과주스, 플레인_휘낭시에, 무화과_휘낭시에)",3


In [ ]:
from mlxtend.frequent_patterns import association_rules
association_df = association_rules(itemset, metric="lift", min_threshold= 1)
association_df.sort_values(by = "lift", ascending = False)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
126,(시나몬_휘낭시에),"(무화과_휘낭시에, (I)패션프루트에이드)",0.187793,0.009390,0.005634,0.030000,3.195000,0.003870,1.021248,0.845857
125,"(무화과_휘낭시에, (I)패션프루트에이드)",(시나몬_휘낭시에),0.009390,0.187793,0.005634,0.600000,3.195000,0.003870,2.030516,0.693523
75,(시나몬_휘낭시에),"(플레인_휘낭시에, 무화과_휘낭시에, 바스크_치즈케이크)",0.187793,0.016901,0.009390,0.050000,2.958333,0.006216,1.034841,0.815029
68,"(플레인_휘낭시에, 무화과_휘낭시에, 바스크_치즈케이크)",(시나몬_휘낭시에),0.016901,0.187793,0.009390,0.555556,2.958333,0.006216,1.827465,0.673352
90,((I)문경선암리사과주스),((H)카모마일),0.123005,0.029108,0.008451,0.068702,2.360256,0.004870,1.042515,0.657150
...,...,...,...,...,...,...,...,...,...,...
40,((I)문경선암리사과주스),((I)패션프루트에이드),0.123005,0.108920,0.014085,0.114504,1.051263,0.000687,1.006306,0.055603
128,"((I)제주유기농귤피주스, 무화과_휘낭시에)",(시나몬_휘낭시에),0.029108,0.187793,0.005634,0.193548,1.030645,0.000168,1.007136,0.030625
131,(시나몬_휘낭시에),"((I)제주유기농귤피주스, 무화과_휘낭시에)",0.187793,0.029108,0.005634,0.030000,1.030645,0.000168,1.000920,0.036609
120,"(브라우니, 잠봉뵈르)",(바스크_치즈케이크),0.017840,0.307042,0.005634,0.315789,1.028489,0.000156,1.012784,0.028203


## 주중/주말

In [ ]:
offline_df['카테고리2'] = offline_df['카테고리'].replace({'시그니처' : '시그니처', 'Basic_ice' : '베이직', '에스프레소' : '에스프레소', 'Basic' : '베이직'})

offline_df2 = offline_df[offline_df['카테고리2'].isin(['에스프레소', '비버리지', '시그니처', '디저트', '베이직'])].copy()

### 주중

In [ ]:
offline_df2.head(1)

,결제일,결제시간,결제내역,합계,상품별 할인,결제 할인,카드 결제,현금 결제,간편 결제,기타 결제,...,day,hour,day_name,year_month,is_weekend,season,is_holiday,weekend_n_holiday,상품명_원산지,카테고리2
0,2022-02-07,19:28:53,샘플 캐모마일,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7,19,Monday,2022-02,0,1,0,0,캐모마일_샘플,에스프레소


In [ ]:
offline_df2 = offline_df2[offline_df2['weekend_n_holiday'] == 0].reset_index(drop = True)

### 1. 휘낭시에 쌍 포함

In [ ]:
from matplotlib.colors import LinearSegmentedColormap
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

In [ ]:
records = offline_df2[["결제일시", "상품명", "카테고리"]]
records = records.dropna()
records.loc[records["카테고리"] == "핸드드립", "상품명"] = records.loc[records["카테고리"] == "핸드드립", "상품명"].apply(lambda x : re.sub(r"\(H\)|\(I\)", "", x ))
records.loc[records["카테고리"] == "핸드드립", "상품명"] = records.loc[records["카테고리"] == "핸드드립", "상품명"].apply(lambda x : x.split("_")[0])
records.loc[records["카테고리"] == "핸드드립", "상품명"].value_counts()


records = records.groupby("결제일시").agg({"상품명" : lambda x : list(x)}).reset_index(drop = True)


records = records[records["상품명"].apply(lambda x : True if len(x) > 1 else False)]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
te = TransactionEncoder()
te_ary = te.fit_transform(records["상품명"])
te_df = pd.DataFrame(te_ary, columns= te.columns_)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
itemset = apriori(te_df,
                  min_support=0.005,
                  max_len=5,
                  use_colnames=True,
                  verbose=1,
                 )
itemset['length'] = itemset['itemsets'].map(lambda x: len(x))
itemset = itemset.sort_values(by = 'support',ascending=False)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Processing 12 combinations | Sampling itemset size 4


In [ ]:
from mlxtend.frequent_patterns import association_rules
association_df = association_rules(itemset, metric="lift", min_threshold= 1)
association_df.sort_values(by = "lift", ascending = False)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
10,(무화과_휘낭시에),(플레인_휘낭시에),0.044386,0.044699,0.013819,0.311346,6.965441,0.011835,1.387200,0.896214
11,(플레인_휘낭시에),(무화과_휘낭시에),0.044699,0.044386,0.013819,0.309170,6.965441,0.011835,1.383284,0.896507
33,((H)아메리카노_쥬시),((H)아메리카노_클래식),0.035993,0.090842,0.008354,0.232104,2.555040,0.005084,1.183960,0.631341
32,((H)아메리카노_클래식),((H)아메리카노_쥬시),0.090842,0.035993,0.008354,0.091964,2.555040,0.005084,1.061639,0.669429
14,(카페_루이지),(슈퍼클린_클래식),0.054575,0.102553,0.012883,0.236052,2.301749,0.007286,1.174748,0.598194
15,(슈퍼클린_클래식),(카페_루이지),0.102553,0.054575,0.012883,0.125619,2.301749,0.007286,1.081250,0.630174
26,((H)아메리카노_클래식),((H)카페라떼_클래식),0.090842,0.042200,0.008627,0.094972,2.250513,0.004794,1.058310,0.611177
27,((H)카페라떼_클래식),((H)아메리카노_클래식),0.042200,0.090842,0.008627,0.204440,2.250513,0.004794,1.142791,0.580139
35,(슈퍼클린_쥬시),(카페_루이지),0.082488,0.054575,0.008237,0.099858,1.829730,0.003735,1.050306,0.494240
34,(카페_루이지),(슈퍼클린_쥬시),0.054575,0.082488,0.008237,0.150930,1.829730,0.003735,1.080609,0.479648


In [ ]:
offline_dfset= offline_df2[['상품명', '상품재분류', '카테고리2']].drop_duplicates()

repl_dict = {}

for x, y in offline_dfset[['상품명', '카테고리2']].values:
    repl_dict[x] = y

res = []
for x in association_df['antecedents'] :
    tmp = []
    for t in x :
        tmp.append(repl_dict[t])
    res.append('_'.join(tmp))
len(res)
association_df['antecedents_cat'] = res

res = []
for x in association_df['consequents'] :
    tmp = []
    for t in x :
        tmp.append(repl_dict[t])

    res.append('_'.join(tmp))
association_df['consequents_cat'] = res

association_df['cat'] = association_df['antecedents_cat']+'_'+association_df['consequents_cat']

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


KeyError: ignored

In [ ]:
select_zip = {'커피류' : ['베이직_베이직', '에스프레소_에스프레소', '에스프레소_시그니처', '시그니처_에스프레소', '시그니처_시그니처'],
 '커피류+디저트' : ['시그니처_디저트_시그니처', '시그니처_시그니처_디저트', '시그니처_디저트_디저트', '베이직_디저트', '디저트_베이직',
             '디저트_시그니처_디저트', '디저트_시그니처', '시그니처_디저트', '디저트_시그니처_시그니처'],
 '시그니처' : ['시그니처_시그니처'],
 '시그니처+디저트' : ['시그니처_디저트_시그니처', '시그니처_시그니처_디저트', '시그니처_디저트_디저트',
             '디저트_시그니처_디저트', '디저트_시그니처', '시그니처_디저트', '디저트_시그니처_시그니처'],
 '에스프레소' : ['에스프레소_에스프레소'],
 '에스프레소+디저트' : [],
 '베이직' : ['베이직_베이직'],
 '베이직+디저트' : ['베이직_디저트', '디저트_베이직'],
 '비버리지' : [],
 '비버리지+디저트' : ['비버리지_디저트', '디저트_비버리지']}

In [ ]:
# 커피류
association_df[association_df['cat'].isin(select_zip['커피류'])][['antecedents', 'consequents',
                                                               'support', 'confidence', 'lift']]

In [ ]:
# 커피류+디저트
association_df[association_df['cat'].isin(select_zip['커피류+디저트'])][['antecedents', 'consequents',
                                                               'support', 'confidence', 'lift']]

In [ ]:
# 시그니처
association_df[association_df['cat'].isin(select_zip['시그니처'])][['antecedents', 'consequents',
                                                               'support', 'confidence', 'lift']]

In [ ]:
# 시그니처+디저트
association_df[association_df['cat'].isin(select_zip['시그니처+디저트'])][['antecedents', 'consequents',
                                                               'support', 'confidence', 'lift']]

In [ ]:
# 에스프레소
association_df[association_df['cat'].isin(select_zip['에스프레소'])][['antecedents', 'consequents',
                                                               'support', 'confidence', 'lift']]

In [ ]:
# 에스프레소+디저트
association_df[association_df['cat'].isin(select_zip['에스프레소+디저트'])][['antecedents', 'consequents',
                                                               'support', 'confidence', 'lift']]

In [ ]:
# 베이직
association_df[association_df['cat'].isin(select_zip['베이직'])][['antecedents', 'consequents',
                                                               'support', 'confidence', 'lift']]

In [ ]:
# 베이직+디저트
association_df[association_df['cat'].isin(select_zip['베이직+디저트'])][['antecedents', 'consequents',
                                                               'support', 'confidence', 'lift']]

In [ ]:
# 비버리지
association_df[association_df['cat'].isin(select_zip['비버리지'])][['antecedents', 'consequents',
                                                               'support', 'confidence', 'lift']]

In [ ]:
# 비버리지+디저트
association_df[association_df['cat'].isin(select_zip['비버리지+디저트'])][['antecedents', 'consequents',
                                                               'support', 'confidence', 'lift']]

### 2. 휘낭시에 쌍 제거

In [ ]:
records = offline_df2[offline_df2['상품재분류'] != '휘낭시에'][["결제일시", "상품명", "카테고리"]]
records = records.dropna()
records.loc[records["카테고리"] == "핸드드립", "상품명"] = records.loc[records["카테고리"] == "핸드드립", "상품명"].apply(lambda x : re.sub(r"\(H\)|\(I\)", "", x ))
records.loc[records["카테고리"] == "핸드드립", "상품명"] = records.loc[records["카테고리"] == "핸드드립", "상품명"].apply(lambda x : x.split("_")[0])
records.loc[records["카테고리"] == "핸드드립", "상품명"].value_counts()


records = records.groupby("결제일시").agg({"상품명" : lambda x : list(x)}).reset_index(drop = True)


records = records[records["상품명"].apply(lambda x : True if len(x) > 1 else False)]

In [ ]:
te = TransactionEncoder()
te_ary = te.fit_transform(records["상품명"])
te_df = pd.DataFrame(te_ary, columns= te.columns_)

In [ ]:
itemset = apriori(te_df,
                  min_support=0.005,
                  max_len=5,
                  use_colnames=True,
                  verbose=1,
                 )
itemset['length'] = itemset['itemsets'].map(lambda x: len(x))
itemset = itemset.sort_values(by = 'support',ascending=False)

In [ ]:
from mlxtend.frequent_patterns import association_rules
association_df = association_rules(itemset, metric="lift", min_threshold= 1)
association_df.sort_values(by = "lift", ascending = False)

In [ ]:
offline_dfset= offline_df2[['상품명', '상품재분류', '카테고리2']].drop_duplicates()

repl_dict = {}

for x, y in offline_dfset[['상품명', '카테고리2']].values:
    repl_dict[x] = y

res = []
for x in association_df['antecedents'] :
    tmp = []
    for t in x :
        tmp.append(repl_dict[t])
    res.append('_'.join(tmp))
len(res)
association_df['antecedents_cat'] = res

res = []
for x in association_df['consequents'] :
    tmp = []
    for t in x :
        tmp.append(repl_dict[t])

    res.append('_'.join(tmp))
association_df['consequents_cat'] = res

association_df['cat'] = association_df['antecedents_cat']+'_'+association_df['consequents_cat']

In [ ]:
## 전체

select_zip = {'커피류' : ['베이직_베이직', '에스프레소_에스프레소', '에스프레소_시그니처', '시그니처_에스프레소', '시그니처_시그니처'],
 '커피류+디저트' : ['시그니처_디저트_시그니처', '시그니처_시그니처_디저트', '시그니처_디저트_디저트', '베이직_디저트', '디저트_베이직',
             '디저트_시그니처_디저트', '디저트_시그니처', '시그니처_디저트', '디저트_시그니처_시그니처'],
 '시그니처' : ['시그니처_시그니처'],
 '시그니처+디저트' : ['시그니처_디저트_시그니처', '시그니처_시그니처_디저트', '시그니처_디저트_디저트',
             '디저트_시그니처_디저트', '디저트_시그니처', '시그니처_디저트', '디저트_시그니처_시그니처'],
 '에스프레소' : ['에스프레소_에스프레소'],
 '에스프레소+디저트' : [],
 '베이직' : ['베이직_베이직'],
 '베이직+디저트' : ['베이직_디저트', '디저트_베이직'],
 '비버리지' : [],
 '비버리지+디저트' : ['비버리지_디저트', '디저트_비버리지']}

## 커피류
association_df[association_df['cat'].isin(select_zip['커피류'])][['antecedents', 'consequents',
                                                               'support', 'confidence', 'lift']]

In [ ]:
# 커피류+디저트
association_df[association_df['cat'].isin(select_zip['커피류+디저트'])][['antecedents', 'consequents',
                                                               'support', 'confidence', 'lift']]


In [ ]:
# 시그니처
association_df[association_df['cat'].isin(select_zip['시그니처'])][['antecedents', 'consequents',
                                                               'support', 'confidence', 'lift']]


In [ ]:
# 시그니처+디저트
association_df[association_df['cat'].isin(select_zip['시그니처+디저트'])][['antecedents', 'consequents',
                                                               'support', 'confidence', 'lift']]


In [ ]:
# 에스프레소
association_df[association_df['cat'].isin(select_zip['에스프레소'])][['antecedents', 'consequents',
                                                               'support', 'confidence', 'lift']]


In [ ]:
# 에스프레소+디저트
association_df[association_df['cat'].isin(select_zip['에스프레소+디저트'])][['antecedents', 'consequents',
                                                               'support', 'confidence', 'lift']]


In [ ]:
# 베이직
association_df[association_df['cat'].isin(select_zip['베이직'])][['antecedents', 'consequents',
                                                               'support', 'confidence', 'lift']]


In [ ]:
# 베이직+디저트
association_df[association_df['cat'].isin(select_zip['베이직+디저트'])][['antecedents', 'consequents',
                                                               'support', 'confidence', 'lift']]


In [ ]:
# 비버리지
association_df[association_df['cat'].isin(select_zip['비버리지'])][['antecedents', 'consequents',
                                                               'support', 'confidence', 'lift']]


In [ ]:
# 비버리지+디저트
association_df[association_df['cat'].isin(select_zip['비버리지+디저트'])][['antecedents', 'consequents',
                                                               'support', 'confidence', 'lift']]

### 주말

In [ ]:
offline_df = offline_df[offline_df['is_weekend'] == 1].reset_index(drop = True)

### 1. 휘낭시에 쌍 포함

In [ ]:
from matplotlib.colors import LinearSegmentedColormap
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

In [ ]:
records = offline_df2[["결제일시", "상품명", "카테고리"]]
records = records.dropna()
records.loc[records["카테고리"] == "핸드드립", "상품명"] = records.loc[records["카테고리"] == "핸드드립", "상품명"].apply(lambda x : re.sub(r"\(H\)|\(I\)", "", x ))
records.loc[records["카테고리"] == "핸드드립", "상품명"] = records.loc[records["카테고리"] == "핸드드립", "상품명"].apply(lambda x : x.split("_")[0])
records.loc[records["카테고리"] == "핸드드립", "상품명"].value_counts()


records = records.groupby("결제일시").agg({"상품명" : lambda x : list(x)}).reset_index(drop = True)


records = records[records["상품명"].apply(lambda x : True if len(x) > 1 else False)]

In [ ]:
te = TransactionEncoder()
te_ary = te.fit_transform(records["상품명"])
te_df = pd.DataFrame(te_ary, columns= te.columns_)

In [ ]:
itemset = apriori(te_df,
                  min_support=0.005,
                  max_len=5,
                  use_colnames=True,
                  verbose=1,
                 )
itemset['length'] = itemset['itemsets'].map(lambda x: len(x))
itemset = itemset.sort_values(by = 'support',ascending=False)

In [ ]:
from mlxtend.frequent_patterns import association_rules
association_df = association_rules(itemset, metric="lift", min_threshold= 1)
association_df.sort_values(by = "lift", ascending = False)

In [ ]:
offline_dfset= offline_df2[['상품명', '상품재분류', '카테고리2']].drop_duplicates()

repl_dict = {}

for x, y in offline_dfset[['상품명', '카테고리2']].values:
    repl_dict[x] = y

res = []
for x in association_df['antecedents'] :
    tmp = []
    for t in x :
        tmp.append(repl_dict[t])
    res.append('_'.join(tmp))
len(res)
association_df['antecedents_cat'] = res

res = []
for x in association_df['consequents'] :
    tmp = []
    for t in x :
        tmp.append(repl_dict[t])

    res.append('_'.join(tmp))
association_df['consequents_cat'] = res

association_df['cat'] = association_df['antecedents_cat']+'_'+association_df['consequents_cat']

In [ ]:
select_zip = {'커피류' : ['베이직_베이직', '에스프레소_에스프레소', '에스프레소_시그니처', '시그니처_에스프레소', '시그니처_시그니처'],
 '커피류+디저트' : ['시그니처_디저트_시그니처', '시그니처_시그니처_디저트', '시그니처_디저트_디저트', '베이직_디저트', '디저트_베이직',
             '디저트_시그니처_디저트', '디저트_시그니처', '시그니처_디저트', '디저트_시그니처_시그니처'],
 '시그니처' : ['시그니처_시그니처'],
 '시그니처+디저트' : ['시그니처_디저트_시그니처', '시그니처_시그니처_디저트', '시그니처_디저트_디저트',
             '디저트_시그니처_디저트', '디저트_시그니처', '시그니처_디저트', '디저트_시그니처_시그니처'],
 '에스프레소' : ['에스프레소_에스프레소'],
 '에스프레소+디저트' : [],
 '베이직' : ['베이직_베이직'],
 '베이직+디저트' : ['베이직_디저트', '디저트_베이직'],
 '비버리지' : [],
 '비버리지+디저트' : ['비버리지_디저트', '디저트_비버리지']}

In [ ]:
# 커피류
association_df[association_df['cat'].isin(select_zip['커피류'])][['antecedents', 'consequents',
                                                               'support', 'confidence', 'lift']]

In [ ]:
# 커피류+디저트
association_df[association_df['cat'].isin(select_zip['커피류+디저트'])][['antecedents', 'consequents',
                                                               'support', 'confidence', 'lift']]

In [ ]:
# 시그니처
association_df[association_df['cat'].isin(select_zip['시그니처'])][['antecedents', 'consequents',
                                                               'support', 'confidence', 'lift']]

In [ ]:
# 시그니처+디저트
association_df[association_df['cat'].isin(select_zip['시그니처+디저트'])][['antecedents', 'consequents',
                                                               'support', 'confidence', 'lift']]

In [ ]:
# 에스프레소
association_df[association_df['cat'].isin(select_zip['에스프레소'])][['antecedents', 'consequents',
                                                               'support', 'confidence', 'lift']]

In [ ]:
# 에스프레소+디저트
association_df[association_df['cat'].isin(select_zip['에스프레소+디저트'])][['antecedents', 'consequents',
                                                               'support', 'confidence', 'lift']]

In [ ]:
# 베이직
association_df[association_df['cat'].isin(select_zip['베이직'])][['antecedents', 'consequents',
                                                               'support', 'confidence', 'lift']]

In [ ]:
# 베이직+디저트
association_df[association_df['cat'].isin(select_zip['베이직+디저트'])][['antecedents', 'consequents',
                                                               'support', 'confidence', 'lift']]

In [ ]:
# 비버리지
association_df[association_df['cat'].isin(select_zip['비버리지'])][['antecedents', 'consequents',
                                                               'support', 'confidence', 'lift']]

In [ ]:
# 비버리지+디저트
association_df[association_df['cat'].isin(select_zip['비버리지+디저트'])][['antecedents', 'consequents',
                                                               'support', 'confidence', 'lift']]

### 2. 휘낭시에 쌍 제거

In [ ]:
records = offline_df2[offline_df2['상품재분류'] != '휘낭시에'][["결제일시", "상품명", "카테고리"]]
records = records.dropna()
records.loc[records["카테고리"] == "핸드드립", "상품명"] = records.loc[records["카테고리"] == "핸드드립", "상품명"].apply(lambda x : re.sub(r"\(H\)|\(I\)", "", x ))
records.loc[records["카테고리"] == "핸드드립", "상품명"] = records.loc[records["카테고리"] == "핸드드립", "상품명"].apply(lambda x : x.split("_")[0])
records.loc[records["카테고리"] == "핸드드립", "상품명"].value_counts()


records = records.groupby("결제일시").agg({"상품명" : lambda x : list(x)}).reset_index(drop = True)


records = records[records["상품명"].apply(lambda x : True if len(x) > 1 else False)]

In [ ]:
te = TransactionEncoder()
te_ary = te.fit_transform(records["상품명"])
te_df = pd.DataFrame(te_ary, columns= te.columns_)

In [ ]:
itemset = apriori(te_df,
                  min_support=0.005,
                  max_len=5,
                  use_colnames=True,
                  verbose=1,
                 )
itemset['length'] = itemset['itemsets'].map(lambda x: len(x))
itemset = itemset.sort_values(by = 'support',ascending=False)

In [ ]:
from mlxtend.frequent_patterns import association_rules
association_df = association_rules(itemset, metric="lift", min_threshold= 1)
association_df.sort_values(by = "lift", ascending = False)

In [ ]:
offline_dfset= offline_df2[['상품명', '상품재분류', '카테고리2']].drop_duplicates()

repl_dict = {}

for x, y in offline_dfset[['상품명', '카테고리2']].values:
    repl_dict[x] = y

res = []
for x in association_df['antecedents'] :
    tmp = []
    for t in x :
        tmp.append(repl_dict[t])
    res.append('_'.join(tmp))
len(res)
association_df['antecedents_cat'] = res

res = []
for x in association_df['consequents'] :
    tmp = []
    for t in x :
        tmp.append(repl_dict[t])

    res.append('_'.join(tmp))
association_df['consequents_cat'] = res

association_df['cat'] = association_df['antecedents_cat']+'_'+association_df['consequents_cat']

In [ ]:
# 커피류
association_df[association_df['cat'].isin(select_zip['커피류'])][['antecedents', 'consequents',
                                                               'support', 'confidence', 'lift']]

In [ ]:
# 커피류+디저트
association_df[association_df['cat'].isin(select_zip['커피류+디저트'])][['antecedents', 'consequents',
                                                               'support', 'confidence', 'lift']]


In [ ]:
# 시그니처
association_df[association_df['cat'].isin(select_zip['시그니처'])][['antecedents', 'consequents',
                                                               'support', 'confidence', 'lift']]


In [ ]:
# 시그니처+디저트
association_df[association_df['cat'].isin(select_zip['시그니처+디저트'])][['antecedents', 'consequents',
                                                               'support', 'confidence', 'lift']]


In [ ]:
# 에스프레소
association_df[association_df['cat'].isin(select_zip['에스프레소'])][['antecedents', 'consequents',
                                                               'support', 'confidence', 'lift']]


In [ ]:
# 에스프레소+디저트
association_df[association_df['cat'].isin(select_zip['에스프레소+디저트'])][['antecedents', 'consequents',
                                                               'support', 'confidence', 'lift']]


In [ ]:
# 베이직
association_df[association_df['cat'].isin(select_zip['베이직'])][['antecedents', 'consequents',
                                                               'support', 'confidence', 'lift']]


In [ ]:
# 베이직+디저트
association_df[association_df['cat'].isin(select_zip['베이직+디저트'])][['antecedents', 'consequents',
                                                               'support', 'confidence', 'lift']]


In [ ]:
# 비버리지
association_df[association_df['cat'].isin(select_zip['비버리지'])][['antecedents', 'consequents',
                                                               'support', 'confidence', 'lift']]


In [ ]:
# 비버리지+디저트
association_df[association_df['cat'].isin(select_zip['비버리지+디저트'])][['antecedents', 'consequents',
                                                               'support', 'confidence', 'lift']]